In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import os
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

In [2]:
y=pd.DataFrame([True,False])

In [3]:
print(y.astype(int))

   0
0  1
1  0


In [4]:
df=pd.read_csv('train.csv')
x_test=pd.read_csv('test.csv')
ids=x_test['PassengerId']


In [5]:
df.keys()

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [6]:
df.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:

#split id
df['PassengerId']=df['PassengerId'].str.split('_')
df['group'] = df['PassengerId'].str.get(0)
df['passenger_num']=df['PassengerId'].str.get(1)
df['group'] = df['group'].astype(int)
df['passenger_num']=df['passenger_num'].astype(int)

#split id test
x_test['PassengerId']=x_test['PassengerId'].str.split('_')
x_test['group'] = x_test['PassengerId'].str.get(0)
x_test['passenger_num']=x_test['PassengerId'].str.get(1)
x_test['group'] = x_test['group'].astype(int)
x_test['passenger_num']=x_test['passenger_num'].astype(int)

In [8]:
#fill missing data
imputer = SimpleImputer(strategy='most_frequent')

# Fit the imputer on your data (calculates the mean for imputation)
imputer.fit(df[['HomePlanet','Destination']])
df[['HomePlanet','Destination']] = imputer.transform(df[['HomePlanet','Destination']])
x_test[['HomePlanet','Destination']] = imputer.transform(x_test[['HomePlanet','Destination']])



In [9]:
#turn to 0,1
df['VIP']=df['VIP'].apply(lambda val:1 if val==True else 0)
x_test['VIP']=x_test['VIP'].apply(lambda val:1 if val==True else 0)

df['CryoSleep']=df['CryoSleep'].apply(lambda val:1 if val==True else 0)
x_test['CryoSleep']=x_test['CryoSleep'].apply(lambda val:1 if val==True else 0)

df['Transported']=df['Transported'].apply(lambda val:1 if val==True else 0)

#fill the other missing data with 0
df['Age'] = df['Age'].fillna(df['Age'].mean())
x_test['Age'] = x_test['Age'].fillna(df['Age'].mean())

df=df.fillna(0)
x_test=x_test.fillna(0)

df=df.drop(['PassengerId','Cabin','Name'],axis=1)
x_test=x_test.drop(['PassengerId','Cabin','Name'],axis=1)

#label encodin
df = pd.get_dummies(df, columns=['HomePlanet','Destination'], prefix=['HomePlanet','Destination'],dtype=int)
x_test = pd.get_dummies(x_test, columns=['HomePlanet','Destination'], prefix=['HomePlanet','Destination'],dtype=int)

#y
y_train=df['Transported']
df=df.drop('Transported',axis=1)


In [10]:
df.head(5)

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,group,passenger_num,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39.0,0,0.0,0.0,0.0,0.0,0.0,1,1,0,1,0,0,0,1
1,0,24.0,0,109.0,9.0,25.0,549.0,44.0,2,1,1,0,0,0,0,1
2,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,3,1,0,1,0,0,0,1
3,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,3,2,0,1,0,0,0,1
4,0,16.0,0,303.0,70.0,151.0,565.0,2.0,4,1,1,0,0,0,0,1


In [11]:
#split the data
x_train, x_cv, y_train, y_cv = train_test_split(df, y_train,test_size=0.20,random_state=32)

In [12]:


#scaling the data using standard scaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])
x_test=scaler.transform(x_test[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])
x_cv=scaler.transform(x_cv[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])


# Create polynomial features (degree 2)
poly = PolynomialFeatures(3, include_bias=False)
x_train=poly.fit_transform(x_train)
x_cv=poly.transform(x_cv)
x_test=poly.transform(x_test)

In [13]:
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [14]:
print(model.score(x_train,y_train))
print(model.score(x_cv,y_cv))

0.7917745182628703
0.78205865439908


In [15]:
'''y_pred=model.predict(x_test)
y_pred=y_pred.astype(bool)

ids=list(ids)
submit=pd.DataFrame({'PassengerId':ids,'Transported':y_pred})
submit.to_csv('submit5.csv',index=False)'''


"y_pred=model.predict(x_test)\ny_pred=y_pred.astype(bool)\n\nids=list(ids)\nsubmit=pd.DataFrame({'PassengerId':ids,'Transported':y_pred})\nsubmit.to_csv('submit5.csv',index=False)"